In [1]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.get_logger().setLevel(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

2024-01-04 23:54:30.431944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 23:54:30.431988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 23:54:30.433324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 23:54:30.439583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 23:54:31.151001: W tensorflow/compiler/tf2

In [3]:
import keras_nlp

In [4]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets  import load_dataset


/home/vrsa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import logging
logging.getLogger("external.local_xla.xla.stream_executor.cuda").setLevel(logging.ERROR)
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.get_logger().setLevel(logging.WARN)


In [6]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vrsa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [8]:
import re
from nltk.corpus import stopwords

def clean_text(text):
    text = text.lower()

    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)

    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\\', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'-:,', '', text)
    text = re.sub(r'\.\b', '', text)
    text = re.sub(r'--|,', '', text)
    text = re.sub(r'\b\b', '', text)
    text = re.sub(r'\b\.\B', '', text)

    text = text.split()
    new_text = []
    for word in text:
        if word in contractions:
            new_text.append(contractions[word])
        else:
            new_text.append(word)
    text = " ".join(new_text)

   


    

    return text


In [9]:
model_path = "t5-small"
data_path = "cnn_dailymail", "3.0.0"

In [10]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [11]:
train_dataset = dataset["train"]

total_examples = len(train_dataset)

half_index = total_examples // 2
third_index = total_examples // 3

# dataset prevelik za moje računalo (thats  what she said)
train_dataset = train_dataset.select(range(third_index))

In [12]:
example = train_dataset[0]  
print(example)

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [14]:
train_dataset = train_dataset.map(lambda example: {'article': clean_text(example['article']),
                                                    'highlights': clean_text(example['highlights'])})


Parameter 'function'=<function <lambda> at 0x7f94a12d32e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 95704/95704 [00:50<00:00, 1883.65 examples/s]


In [15]:
example = train_dataset[0]  
print(example)

{'article': 'london england (reuters) harry potter star daniel radcliffe gains access reported £20 million ($411 million) fortune turns 18 monday insists money cast spell him daniel radcliffe harry potter "harry potter order phoenix" disappointment gossip columnists around world young actor says plans fritter cash away fast cars drink celebrity parties "i plan one people who soon turn 18 suddenly buy massive sports car collection something similar" told australian interviewer earlier month "i think i ll particularly extravagant "the things like buying things cost 10 pounds books cds dvds" 18 radcliffe able gamble casino buy drink pub see horror film "hostel: part ii" currently six places number one movie uk box office chart details he ll mark landmark birthday wraps agent publicist comment plans "i ll definitely sort party" said interview "hopefully none reading it" radcliffe s earnings first five potter films held trust fund able touch despite growing fame riches actor says keeping fe

In [16]:
def tokenization(examples):
    inputs = ['summarize: ' + doc for doc in examples["article"]]
    tok = tokenizer(inputs, truncation=True, padding = True, return_tensors='tf')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'],truncation=True, padding = True, return_tensors='tf')

    tok['labels'] = labels['input_ids']

    return {'input_ids': tok['input_ids'].numpy(),
            'labels': tok['labels'].numpy(),
            'attention_mask': tok['attention_mask'].numpy()}

In [17]:
tokenized_dataset = train_dataset.map(tokenization, batched=True)

Map:   0%|          | 0/95704 [00:00<?, ? examples/s]2024-01-04 23:55:41.083601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-04 23:55:41.084719: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-04 23:55:41.084920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/bl

In [18]:
processed_data = tokenized_dataset.remove_columns(['article','highlights', 'id'])

In [19]:
processed_data

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 95704
})

In [20]:
final_dataset = processed_data.train_test_split(
    train_size=0.25, test_size=0.05
)
final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 23926
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 4786
    })
})

In [21]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)

2024-01-04 23:57:44.627905: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [22]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model= model, return_tensors="tf")

In [24]:
train_dataset = model.prepare_tf_dataset(final_dataset['train'], batch_size=1, tokenizer= tokenizer, collate_fn=data_collator, shuffle=True, drop_remainder=True)

/home/vrsa/.local/lib/python3.10/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [25]:
test_dataset = model.prepare_tf_dataset(final_dataset['test'], batch_size=2, tokenizer= tokenizer, collate_fn=data_collator, shuffle=False, drop_remainder=True)

In [26]:
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [27]:
rouge_l = keras_nlp.metrics.RougeL()

In [28]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    result = {"RougeL": result["f1_score"]}

    return result

In [29]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=test_dataset)

In [30]:
callbacks = [metric_callback]

In [31]:
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3


2024-01-04 23:57:58.873707: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9362e82db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-04 23:57:58.873736: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-01-04 23:57:58.879064: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-04 23:57:58.892305: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1704409078.944206   33544 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23926/23926 [==============================] - 2347s 97ms/step - loss: 1.2685 - val_loss: 1.0977
Epoch 2/3
23926/23926 [==============================] - 2335s 98ms/step - loss: 1.1745 - val_loss: 1.0856
Epoch 3/3
23926/23926 [==============================] - 2333s 98ms/step - loss: 1.1477 - val_loss: 1.0737


In [44]:
#model.save("t5_model.h5")

In [34]:
from transformers import pipeline

In [35]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

In [38]:
dataset['train'][12]['article']

'BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of  7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanú

In [40]:
input_text = dataset['train'][12]['article']

summary = pipe(input_text)


In [41]:
summary

[{'summary_text': "Carlos Alberto joins Werder Bremen for a club record fee $10 million . 22-year-old scored in FC Porto's champions league final win against Monaco 2004 . he started career with fluminense and helped lift campeonato ."}]

In [43]:
article_summaries = {}

for i in range(10):
    article_text = dataset['train'][i]['article']
    
    summary = pipe(article_text)
    
    article_summaries[f"Article_{i+1}"] = {
        'article': article_text,
        'summary': summary
    }


In [45]:
import json
output_file_path = 'article_summaries.json'
with open(output_file_path, 'w') as json_file:
    json.dump(article_summaries, json_file, indent=2)

In [46]:
file_path = 'index_article.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

print(file_content)

Croatia had the lowest number of births in history last year

Written by: Branimir Perković
22:30, January 4, 2024.
Croatia had the lowest number of births in history last year
Photo: Shutterstock
CROATIA is in a very difficult situation demographically, both due to emigration and the decline in the number of births. These two problems are actually connected, because mostly young people are emigrating who are just starting or in the phase of creating a family, and thus the potential for having children in the country is decreasing.

In terms of demographic problems, Croatia is not unique in the world, but it is one of the most affected countries. The problems with the decline in the number of children born throughout Europe, the astonishing decline in the total number of births in China in recent years, the inability of South Korea and Japan to slow down the rapid aging of the population and the demographic decline in the USA testify that this is a global problem.
AD

Record low number

In [47]:
summary = pipe(file_content)
summary


[{'summary_text': 'julian zelizer: 2023 worst year in history in terms of number of births . says emigration slowing rapid aging population a global problem . he says a positive migration balance may alleviate problems . dexter: a negative migration balance should be added .'}]

In [48]:
file_path = 'index_zivot_na_vagi_hrv.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()


In [49]:
summary = pipe(file_content)
summary

[{'summary_text': 'pobjednik ivota na vagi je Mislav epi +10 Foto: Press U STUDIJU zagrebakog Jadran filma . pokazali su u punom sjaju da su se isplatili zalaganje u proteklih pet mjeseci .'}]

In [50]:
file_path = 'index_zivot_na_vagi_eng.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

In [51]:
summary = pipe(file_content)
summary

[{'summary_text': 'mislav epi weighed 173.9 kilograms five months . he lost 46.9% of her body mass . "it was really worth it" says one contestant .'}]